<a href="https://colab.research.google.com/github/LEEGWONSEOK/MLDL_Basic/blob/main/5_3_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
## train / test 분리
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
## 랜덤포레스트 앙상블 모델

from sklearn.model_selection import cross_validate    # 교차검증
from sklearn.ensemble import RandomForestClassifier   # 랜덤포레스트 모듈

rf = RandomForestClassifier(n_jobs=-1, random_state=42)   # 객체모델 생성, n_jobs로 CPU 전부 사용해서 성능 개선
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)    # n_jobs=-1로 병렬교차검증 진행(성능개선때문에)
# return_train_score = True : 검증점수 뿐만 아니라 train set에 대한 점수도 같이 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 약간 과대적합 나옴!

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)    # feature_importances_ : 특성 중요도

# 좌측부터 알콜함량, 당도, pH 순으로 당도가 높게 나온다

[0.23167441 0.50039841 0.26792718]


In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
# oob_score=True : 자체적으로 부트스트랩 샘플을 만들어서 모델을 평가하는 점수를 얻을 수 있음

rf.fit(train_input, train_target)
print(rf.oob_score_)

# 교차검증과 비슷한 결과가 나옴, OOB 점수 사용하면 교차검증 대신 가능하다

0.8934000384837406


In [6]:
## 엑스트라 트리 모델
# 더 많은 결정트리를 이용하며, 무작위성이 크고, 랜덤하게 노드를 분할하여 계산속도가 빠르다

from sklearn.ensemble import ExtraTreesClassifier   # 엑스트라트리 모델 모듈

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

# 랜덤포레스트보다 당도와 pH의 중요도가 조금 올라간 것을 확인!

[0.20183568 0.52242907 0.27573525]


In [8]:
## 그레이디언트 부스팅
# 경사하강법 스타일이고 깊이가 얕은 결정트리를 사용해 이전트리의 오차를 천천히 줄여주는 방법, 성능은 좋고 속도는 느리고

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 과적합이 거의 없다!

0.8881086892152563 0.8720430147331015


In [9]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
# n_estimators : 결정트리 개수 / 디폴트=100
# learning_rate : 학습률 / 디폴트=0.1
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 과적합이라고 하긴 그렇지만 전반적인 성능이 올라갔다

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

# 여기서는 당도의 중요도가 더 높게 나오네?

[0.15872278 0.68010884 0.16116839]


In [11]:
## 히스토그램 기반 그레이디언트 부스팅

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 성능 개미침 ㅎㅎㅎ

0.9321723946453317 0.8801241948619236


In [12]:
from sklearn.inspection import permutation_importance   # 특성중요도 확인!

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

In [ ]:
## XGBoost

from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

In [ ]:
## LightGBM

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))